In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
import seaborn as sns
sns.set()

In [ ]:
raw_data = pd.read_csv('/kaggle/input/playground-series-s3e20/train.csv')
raw_data.head()

In [ ]:
raw_data.shape

- 79023 oberservations, 76 features

In [ ]:
df = raw_data.copy()

# Handle any missing values

In [ ]:
df_missing_values = pd.DataFrame(data=df.isnull().sum().sort_values(ascending=False),columns=['missing'])
df_missing_values.plot.bar(figsize=(15,3),fontsize=7)

Drop all columns where most of the data is missing.

In [ ]:
columns_to_drop = df_missing_values.query("missing > 20000").index
df = df.drop(columns_to_drop,axis=1)
df.shape

In [ ]:
imputer = SimpleImputer(missing_values=np.nan,strategy='mean')

In [ ]:
imputation = imputer.fit_transform(df.drop('ID_LAT_LON_YEAR_WEEK',axis=1))

In [ ]:
df2 = pd.DataFrame(data=imputation,columns=df.drop('ID_LAT_LON_YEAR_WEEK',axis=1).columns)
df2['ID_LAT_LON_YEAR_WEEK'] = df['ID_LAT_LON_YEAR_WEEK']
df2['year'] = df2['year'].astype('int32')
df2['week_no'] = df2['week_no'].astype('int32')
df2.head(10)

# EDA

### Distribution of data

In [ ]:
fig, axs = plt.subplots(15,5,figsize=(12,50))
fig.subplots_adjust(hspace=0.5)

axs = axs.flatten()

for i, column in enumerate(df.describe().columns):
    axs[i].hist(df[column])
    axs[i].set_title(column,size=6)

In [ ]:
df2['year_week'] = pd.to_datetime(df2['year'].astype(str) + df['week_no'].astype(str) + '0', format='%G%V%w')
df2['LAT_LON'] = df2['latitude'].astype('string') + '_' + df2['longitude'].astype('string')

In [ ]:
df2_loc_mean = df2.groupby('LAT_LON').agg('mean').sort_values('emission',ascending=False)
df2_loc_sum = df2.groupby('LAT_LON').agg('sum').sort_values('emission',ascending=False)

In [ ]:
# simple boxplot with target to find outliers
sns.boxplot(df2, y='emission')

Again we can see that there are some pretty significant outliers. Consider:
* Dropping outliers alltogether
* Replacing outliers with mean
* Keep them if we find that they are part of the natural pattern

### Average emissions by location

In [ ]:
# create subplots structure
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15,5))

# assign plots to subplots
df2_loc_mean.emission.plot.line(ax=ax1)
df2_loc_sum.emission.plot.line(ax=ax2)

# style subplots for readability
ax1.tick_params(axis='x', labelrotation = 45)
ax2.tick_params(axis='x', labelrotation = 45)
ax1.set_title('average emissions by location')
ax2.set_title('total emissions by location')
ax1.set_ylabel('Emissions')

plt.show()

We can see that the vast majority of emissions come from a small number of location. Let's visualize where they are:

### Emissions on the map

In [ ]:
# create subplots
fig, ax = plt.subplots(figsize=(8,6))

# get country from geopandas
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
countries[countries["name"] == "Rwanda"].plot(color="lightgrey",ax=ax)

# plot data on top
df2.plot.scatter(x="longitude",y="latitude", s='emission', c='emission', ax=ax)
ax.grid(alpha=0.5)
plt.show()

### Emissions over time

In [ ]:
# Understand how average emissions fluctuate over the time period
sns.lineplot(data=df2, x='year_week', y='emission', hue='year')

In [ ]:
# Generate unique locations where emissions are unusually high
locations_above_600 = set(df2.query("emission > 600")['LAT_LON'])

# Look at the outlier locations in isolation
for location in locations_above_600:
    df2.query("LAT_LON == @location").plot.line(x='year_week', y='emission',title=location, figsize=(12,4))
    plt.show()

# Feature selection

In [ ]:
# Drop non-numerical features
df3 = df2.drop(['ID_LAT_LON_YEAR_WEEK','emission','year_week'],axis=1)

In [ ]:
# Look at correlation of variables
corrs = df3.corrwith(df2['emission']).sort_values(ascending=False)
corrs.plot.bar(figsize=(15,3),fontsize=7)

In [ ]:
# Top correlated features
corrs.head()

# Modeling

### Without outliers
Since the large outliers are confined to a few locations, removing them will create a more balanced dataset.

In [ ]:
threshold = df2.emission.quantile(0.995)
df_no_outliers = df2.drop(['LAT_LON','year_week','ID_LAT_LON_YEAR_WEEK'],axis=1).query("emission < @threshold")

In [ ]:
X = df_no_outliers.drop('emission',axis=1)
y = df_no_outliers['emission']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=12)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
def fit_estimator(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    score = model.score(X_test, y_test)
    print(model)
    print(f"> Root-mean-squared-error: {rmse}")
    print(f"> Score: {score}")

In [ ]:
tree = DecisionTreeRegressor()
fit_estimator(tree, X_train, y_train, X_test, y_test)

In [ ]:
lin = LinearRegression()
fit_estimator(lin, X_train, y_train, X_test, y_test)

In [ ]:
forest = RandomForestRegressor()
fit_estimator(forest, X_train, y_train, X_test, y_test)

In [ ]:
forest.feature_importances_

In [ ]:
pd.DataFrame(data=forest.feature_importances_,index=X.columns,columns=['importance %'])

### With outliers

In [ ]:
df_with_outliers = df2.drop(['LAT_LON','year_week','ID_LAT_LON_YEAR_WEEK'],axis=1)
X_out = df_with_outliers.drop('emission',axis=1)
y_out = df_with_outliers['emission']

X_train, X_test, y_train, y_test = train_test_split(X_out, y_out, train_size=0.7, shuffle=True, random_state=12)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
tree2 = DecisionTreeRegressor()
fit_estimator(tree2,X_train, y_train, X_test, y_test)

In [ ]:
forest2 = RandomForestRegressor()
fit_estimator(forest2, X_train, y_train, X_test, y_test)

### Only most important features

In [ ]:
df_most = df2[['latitude','longitude','year','week_no','emission']]
X_most = df_most.drop('emission',axis=1)
y_most = df_most['emission']

X_train, X_test, y_train, y_test = train_test_split(X_most, y_most, train_size=0.7, shuffle=True, random_state=12)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
tree3 = DecisionTreeRegressor()
fit_estimator(tree3, X_train, y_train, X_test, y_test)

In [ ]:
df_most_no_outliers = df2[['latitude','longitude','year','week_no','emission']].query("emission < @threshold")
X_mn = df_most_no_outliers.drop('emission',axis=1)
y_mn = df_most_no_outliers['emission']

X_train, X_test, y_train, y_test = train_test_split(X_mn, y_mn, train_size=0.7, shuffle=True, random_state=12)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
tree4 = DecisionTreeRegressor()
fit_estimator(tree4, X_train, y_train, X_test, y_test)